USING ANN MODEL TO PREDICT VALUES OF POWER (OF A POWER PLANT) ASSUMING A NONLINEAR RELATION

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
data = pd.read_excel('Folds5x2_pp.xlsx')
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [10]:
data.columns

Index(['AT', 'V', 'AP', 'RH', 'PE'], dtype='object')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [12]:
data.isnull().sum()

,0
AT,0
V,0
AP,0
RH,0
PE,0


In [13]:
data.nunique()

,0
AT,2773
V,634
AP,2517
RH,4546
PE,4836


In [17]:
y = data['PE']
x = data.drop('PE', axis=1)
y.head()
x.head()


,AT,V,AP,RH
0,14.96,41.76,1024.07,73.17
1,25.18,62.96,1020.04,59.08
2,5.11,39.40,1012.16,92.14
3,20.86,57.32,1010.24,76.64
4,10.82,37.50,1009.23,96.62


In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, shuffle =True )
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size = 0.2, shuffle=True)

In [26]:
print("Shape of the x_train", x_train.shape)
print("Shape of the x_test", x_test.shape)
print("Shape of the x_val", x_val.shape)
print("Shape of the y_train", y_train.shape)
print("Shape of the y_test", y_test.shape)
print("Shape of the y_val", y_val.shape)

Shape of the x_train (6123, 4)
Shape of the x_test (1914, 4)
Shape of the x_val (1531, 4)
Shape of the y_train (6123,)
Shape of the y_test (1914,)
Shape of the y_val (1531,)


In [40]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

# Scale the features
x_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)
x_val_scaled = x_scaler.transform(x_val)

# Scale the target variable
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = y_scaler.transform(y_val.values.reshape(-1, 1))

No activation = 'softmax' for a linear model. Softmax only used for multiclass problems.

In [41]:
regressor = Sequential()
# Adding the input layer and the first hidden layer
regressor.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = x_train_scaled.shape[1]))

# Adding the second hidden layer
regressor.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
regressor.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer with linear activation for regression
regressor.add(Dense(units = 1, kernel_initializer = 'uniform'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['MeanSquaredLogarithmicError'])

# Fitting the ANN to the Training set
model = regressor.fit(x_train_scaled, y_train_scaled, batch_size = 32, epochs = 200,validation_data=(x_val_scaled, y_val_scaled),
              shuffle=True)

Epoch 1/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - MeanSquaredLogarithmicError: 0.1526 - loss: 0.7728 - val_MeanSquaredLogarithmicError: 0.0158 - val_loss: 0.1016
Epoch 2/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MeanSquaredLogarithmicError: 0.0145 - loss: 0.0909 - val_MeanSquaredLogarithmicError: 0.0114 - val_loss: 0.0761
Epoch 3/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MeanSquaredLogarithmicError: 0.0109 - loss: 0.0727 - val_MeanSquaredLogarithmicError: 0.0108 - val_loss: 0.0734
Epoch 4/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MeanSquaredLogarithmicError: 0.0104 - loss: 0.0681 - val_MeanSquaredLogarithmicError: 0.0097 - val_loss: 0.0656
Epoch 5/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - MeanSquaredLogarithmicError: 0.0096 - loss: 0.0638 - val_MeanSquaredLogarithmicError: 0.0096 - val_loss: 0.0633
Epoch 6/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MeanSquaredLogarithmicError: 0.0099 - loss: 0.0628 - val_MeanSquaredLogarithmicError: 0.0096 - val_loss: 0.065

Now we show the actual vs predicted value visually.

In [44]:
y_pred_scaled = regressor.predict(x_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)),1))


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[[475.72 479.17]
 [482.12 476.58]
 [447.68 448.15]
 ...
 [443.39 444.76]
 [447.73 446.34]
 [443.28 451.78]]


In [50]:
y_pred_scaled = regressor.predict(x_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

mae_no = sklearn.metrics.mean_absolute_error(y_test, y_pred)
mse_no = sklearn.metrics.mean_squared_error(y_test, y_pred)
rms    = sqrt(mse_no)

print('Mean Absolute Error     :', mae_no)
print('Mean Square Error       :', mse_no)
print('Root Mean Square Error  :', rms)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Mean Absolute Error     : 3.0698482794846367
Mean Square Error       : 15.568980953703813
Root Mean Square Error  : 3.945754801518185
